
# Write functions that filter and transform dataframes, using provided parameters (PARLA)

## Problem
- Write a function that computes server response times within a given date range
- Write a function that calculates total user revenue during a specified period for users who visited the website during the specified period
- Write a function that calculates total revenue per user for a specified period, and only for users who visited the website at any time before 'end_date'

## Action
- I implemented these functions, using python and pandas

## Result
- The function successfully passed all tests

## Learning
- I revised relevant python and pandas functions

## Application
- I can apply the relevant python and pandas functions for data-related problems


In [14]:

from datetime import datetime
import pandas as pd


def get_response_time(
    df_web_logs: pd.DataFrame,
    begin_date: datetime,
    end_date: datetime
) -> pd.DataFrame:
    """
    Computes server response times within a given date range.

    Filters the `df_web_logs` table by date and returns:
    (1) user_id and (2) load_time, renamed as 'metric'.

    Each request is treated independently, so no grouping by user_id is needed.

    :param df_web_logs: Website log table containing columns ['user_id', 'date', 'load_time'].
    :param begin_date: Start of the filtering period (inclusive).
    :param end_date: End of the filtering period (exclusive).

    :return: DataFrame with columns ['user_id', 'metric'].
    """

    df = df_web_logs[(begin_date <= df_web_logs.date) & (df_web_logs.date < end_date)]

    df = df[['user_id', 'load_time']]

    df = df.rename(columns={'load_time' : 'metric'})

    df = df.reset_index(drop=True)

    return df


def get_revenue_web(
    df_sales: pd.DataFrame,
    df_web_logs: pd.DataFrame,
    begin_date: datetime,
    end_date: datetime
) -> pd.DataFrame:
    """
    Calculates total user revenue during a specified period
    for users who visited the website during the specified period.

    This is useful for analyzing experiments where only users who
    visited the site during the test window are included.

    Returns the total revenue (sum of 'price') per user_id
    for the specified period. If a user visited but made no purchases,
    their total revenue will be zero. There will be exactly one row
    per user_id in the result.

    :param df_sales: Sales data with columns ['user_id', 'date', 'price'].
    :param df_web_logs: Web log data with columns ['user_id', 'date', 'load_time'].
    :param begin_date: Start of the period (inclusive).
    :param end_date: End of the period (exclusive).

    :return: DataFrame with columns ['user_id', 'metric'], where 'metric' is the total revenue.
    """

    # filter dataframe by dates
    logs = df_web_logs[(begin_date <= df_web_logs.date) & (df_web_logs.date < end_date)]
    sales = df_sales[(begin_date <= df_sales.date) & (df_sales.date < end_date)]

    # merge dataframes, subset and rename columns
    merged = pd.merge(logs, sales, how='left', on=['user_id', 'date'])
    merged = merged[['user_id', 'price']]
    merged = merged.rename(columns={'price' : 'metric'})

    # group and calculate total revenue per user
    grouped = merged.groupby(by='user_id').sum().reset_index(drop=False)
    return grouped


def get_revenue_all(
    df_sales: pd.DataFrame,
    df_web_logs: pd.DataFrame,
    begin_date: datetime,
    end_date: datetime
) -> pd.DataFrame:
    """
    Calculates total revenue per user for a specified period,
    and only for users who visited the website at any time before `end_date`.

    This is useful for experiments like email campaigns, where participants
    are users who have previously interacted with the site.

    The function returns a DataFrame with `user_id` and their total purchase
    amount (sum of 'price') between `begin_date` and `end_date`.
    If a user made no purchases in the period, their revenue is zero.
    Each user appears exactly once in the returned DataFrame.

    :param df_sales: Sales table with columns ['user_id', 'date', 'price'].
    :param df_web_logs: Website logs with columns ['user_id', 'date', 'load_time'].
    :param begin_date: Start of the revenue calculation period (inclusive).
    :param end_date: End of the revenue calculation period (exclusive).

    :return: DataFrame with columns ['user_id', 'metric'], where 'metric' is total revenue.
    """

    # select unique users that logged in before 'end_date'
    users = df_web_logs[(df_web_logs.date < end_date)]
    users = users[['user_id']].drop_duplicates().sort_values(by='user_id').reset_index(drop=True)

    # select sales within the specified range
    sales = df_sales[(begin_date <= df_sales.date) & (df_sales.date < end_date)]

    # merge dataframes, subset and rename columns
    merged = pd.merge(users, sales, how='left', on=['user_id'])
    merged = merged[['user_id', 'price']]
    merged = merged.rename(columns={'price' : 'metric'})

    # group by user, calculate total revenue for each user
    grouped = merged.groupby(by='user_id').sum().reset_index(drop=False)
    grouped['metric'] = grouped['metric'].astype(int)
    return grouped



In [15]:

# testing implemented functions

df_sales = pd.DataFrame({
    'date': [datetime(2022, 3, day, 11) for day in range(11, 14)],
    'price': [1100, 900, 1500],
    'user_id': ['1', '2', '1'],
})

df_web_logs = pd.DataFrame({
    'date': [datetime(2022, 3, day, 11) for day in range(10, 14)],
    'load_time': [80.8, 90.1, 15.8, 19.7],
    'user_id': ['3', '1', '2', '1'],
})

begin_date = datetime(2022, 3, 11, 9)
end_date = datetime(2022, 4, 11, 9)

# test 1
answer = pd.DataFrame({'user_id': ['1', '2', '1'], 'metric': [90.1, 15.8, 19.7],})
result = get_response_time(df_web_logs, begin_date, end_date)
print(answer)
print(result)
print(f'test 1: {answer.equals(result)}\n')

# test 2
answer = pd.DataFrame({'user_id': ['1', '2'], 'metric': [2600, 900],})
result = get_revenue_web(df_sales, df_web_logs, begin_date, end_date)
print(answer)
print(result)
print(f'test 2: {answer.equals(result)}\n')

# test 3
answer = pd.DataFrame({'user_id': ['1', '2', '3'], 'metric': [2600, 900, 0],})
result = get_revenue_all(df_sales, df_web_logs, begin_date, end_date)
print(answer)
print(result)
print(f'test 3: {answer.equals(result)}\n')



  user_id  metric
0       1    90.1
1       2    15.8
2       1    19.7
  user_id  metric
0       1    90.1
1       2    15.8
2       1    19.7
test 1: True

  user_id  metric
0       1    2600
1       2     900
  user_id  metric
0       1    2600
1       2     900
test 2: True

  user_id  metric
0       1    2600
1       2     900
2       3       0
  user_id  metric
0       1    2600
1       2     900
2       3       0
test 3: True

